# Train linear model to alphabet sign recognition

In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import Dataset
import torchvision
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import os
from PIL import Image
from tqdm import tqdm
import numpy as np

In [3]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data.iloc[index].values
        return sample

In [4]:
data_path = "D:\\project\\dataset\\gen_land_marks_dataset\\LM_min_max .csv"


data = pd.read_csv(data_path)
data['sign_n'] = data['sign'].astype('category').cat.codes

data

,x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,...,y18,z18,x19,y19,z19,x20,y20,z20,sign,sign_n
0,0.495022,1.0,0.974262,0.828853,0.882998,0.436342,1.0,0.510281,0.408441,0.930107,...,0.450038,0.502614,0.118184,0.622809,0.617609,0.209679,0.693113,0.848838,a,0
1,0.469361,1.0,0.991974,0.819125,0.906074,0.445117,1.0,0.536335,0.404578,0.908042,...,0.454920,0.488348,0.112306,0.636905,0.570991,0.204856,0.676160,0.788405,a,0
2,0.493057,1.0,0.975774,0.835015,0.905123,0.418454,1.0,0.535364,0.372406,0.905587,...,0.472757,0.478108,0.124417,0.643154,0.549644,0.227330,0.688799,0.753638,a,0
3,0.497036,1.0,0.978554,0.829442,0.884008,0.451723,1.0,0.529599,0.374636,0.902809,...,0.466858,0.550284,0.121133,0.631655,0.627232,0.218818,0.672396,0.820793,a,0
4,0.506610,1.0,0.999308,0.848605,0.876080,0.461688,1.0,0.512127,0.425191,0.881576,...,0.465854,0.380969,0.125847,0.632779,0.530544,0.183727,0.672446,0.794712,a,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,0.570976,1.0,1.000000,0.872495,0.895524,0.748421,1.0,0.628247,0.698132,0.697144,...,0.528894,0.000000,0.156402,0.660963,0.280768,0.161146,0.711858,0.537918,z,25
1811,0.628784,1.0,1.000000,0.928527,0.866675,0.428077,1.0,0.618804,0.344258,0.677055,...,0.539672,0.223838,0.160516,0.665505,0.417114,0.198585,0.721275,0.668195,z,25
1812,0.640344,1.0,1.000000,0.900157,0.863415,0.798249,1.0,0.574449,0.689323,0.779147,...,0.519328,0.001633,0.318658,0.647782,0.243037,0.272015,0.710939,0.493679,z,25
1813,0.468633,1.0,1.000000,0.805085,0.886132,0.742283,1.0,0.640918,0.579735,0.871521,...,0.496653,0.069213,0.308547,0.624861,0.221126,0.224839,0.681041,0.434460,z,25


In [27]:
class MyDatasetLinear(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data.iloc[index].values
        sign_data =torch.Tensor(sample[:63].astype(float))
        return sign_data, int(sample[-1])

In [28]:
#train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
dataset = MyDatasetLinear(data)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
t , v = random_split(dataset, [train_size, val_size])

In [30]:

#test_dataset = MyDataset(test_data)
TrainDataLoader = DataLoader(t, batch_size=64)
TestDataLoader = DataLoader(v, batch_size=64)

In [39]:
class LeanarModel(nn.Module):
    def __init__(self, num_classes):
        super(LeanarModel, self).__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(63, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes),
        )
    
    def forward(self, x):
        x = self.classifier(x)
        return x

model = LeanarModel(26)

In [33]:
checkpoint_path = "D:/project/asl/checkpoint_lm_linear.pt"

def save_model(model):
    torch.save(model.state_dict(), checkpoint_path)

In [40]:
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

In [44]:

model.to(device)
opt_myCNN = optim.Adam(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()
num_epochs=150

In [45]:
def validate(model, dataloader):
    #print("validate")
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    val_running_errors = 0
    #print("1")
    with torch.no_grad():
        #print("2")
        for i, data in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
            #print(data[1].shape)
            data, labels = data[0].to(device), data[1].to(device)

            outputs = model(data)
            #print(outputs)
            #print(labels)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()* data.size(0)
            _, preds = torch.max(outputs.data, 1)
            val_running_correct += torch.sum(preds == labels)
            val_running_errors += torch.sum(preds != labels)

        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = val_running_correct/len(dataloader.dataset)
        val_errore = val_running_errors/len(dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy}')
        
        return val_loss, val_accuracy, val_errore
# train part
def train(model,optimizer, dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    train_running_errors = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset) /dataloader.batch_size)):

        data, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(data)

        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs.data, 1)
        train_running_errors += torch.sum(preds != labels)
        train_running_correct += torch.sum(preds == labels)
        train_running_loss += loss.item()* data.size(0)
        loss.backward()
        optimizer.step()
        
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = train_running_correct/len(dataloader.dataset)
    train_errore = train_running_errors/len(dataloader.dataset)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy}")
    return train_loss, train_accuracy, train_errore
 #  main 
def model_train(model, optimizer, train_dataloader ,test_dataloader, epochs):
  err_history = {"train" : [], "val" : []}
  acc_history =  {"train" : [], "val" : []}
  loss_history = {"train" : [], "val" : []}
  for epoch in range(epochs):
      print(f"Epoch {epoch+1} of {epochs}")
      phase = "train"
      print(phase)
      loss, accuracy, error = train(model, optimizer, train_dataloader)
      phase = "val"
      print(phase)
      loss, accuracy, error = validate(model, test_dataloader)
      err_history[phase].append(error)
      acc_history[phase].append(accuracy)
      loss_history[phase].append(loss)
  save_model(model)
  return err_history, acc_history, loss_history

In [46]:
myCNN_err_history, myCNN_acc_history , myCNN_loss_history = model_train(model, opt_myCNN, TrainDataLoader, TestDataLoader ,num_epochs )

Epoch 1 of 150
train


23it [00:00, 66.28it/s]                        


Train Loss: 0.8230, Train Acc: 0.7176308631896973
val


6it [00:00, 92.32it/s]               


Val Loss: 0.5439, Val Acc: 0.8429751992225647
Epoch 2 of 150
train


...

Val Loss: 0.1669, Val Acc: 0.9669421315193176
Epoch 150 of 150
train


23it [00:00, 81.56it/s]                        


Train Loss: 0.1818, Train Acc: 0.9400826692581177
val


6it [00:00, 105.22it/s]              

Val Loss: 0.1631, Val Acc: 0.9641873240470886
